In [1]:
from compChem.Hartree_Fock import Molecule # package is available on github
from scipy.linalg import eigh, eig
import numpy as np
import psi4

In [2]:
numpy_memory = 4
psi4.set_memory(int(5e8))

500000000

In [3]:
class ConstrainedMolecule:
    def __init__(self, geom_file, radical=False):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
    
    def setChargeDensity(self, new_matrix):
        """sets the pMatrix to a new value"""
        self.pMatrix = new_matrix
        
    
    def basischanger(self):
        """
        changes to NO basis, applies constraint, then changes back
        """
        # transform p to MO basis, where mo basis = the eigenfunctions of the f_a operator
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        f_a, f_b = self.id.displayFockMatrix("alpha"), self.id.displayFockMatrix("beta")
        p = (a+b)/2
        c = eigh(f_a, self.id.overlap)[1] # we only need the c matrix, not the eigenvalues themselves,
        
        delta = (f_b-f_a)/2
        f_cs = (f_a+f_b)/2
        # pay attention, c matrices are not unitary
        c_inv = np.linalg.inv(c) # we need the inverse for later
        p_trans = np.einsum("pq, qr, rs->ps", c_inv, p, c_inv.T, optimize=True)
        delta_trans = np.einsum("pq, qr, rs->ps", c.T, delta, c, optimize=True)
        
        
         # transform the fock matrices to NO basis
        d = eigh(p_trans)[1]
        d = d[:, ::-1] #invert all collumns
    
        d_inv = np.linalg.inv(d)
        delta_no = np.einsum("pq, qr, rs->ps", d.T, delta_trans, d, optimize=True)
        
        alpha = self.id.alpha 
        beta = self.id.beta
        #alter first blocks
        
        delta_no[alpha:, :beta] = 0
        delta_no[:beta, alpha:] = 0
        delta_ao = np.einsum("pq, qr, rs->ps",c_inv.T.dot(d_inv.T), delta_no, d_inv.dot(c_inv), optimize=True)
        
        
        return f_cs - delta_ao, f_cs + delta_ao
    
    
    def iteratinator(self):
        """adapted version of the original iterator function in the compChem.Hartree_Fock Molecule object"""
        # setting up entry parameters for the while loop
        E_new = 0  
        E_old = 0
        convergence = False

        # step 2: start iterating
        itercount = 0
        while not convergence and itercount < 5000:
            # calculating block: calculates energies
            E_new = self.id.getElectronicEnergy()
            E_total = self.id.getTotalEnergy()

            # generating block: generates new matrices UHF: account for alpha and beta, first we do the CUHF alteration
            f_a_n, f_b_n = self.basischanger()
            self.id.setGuess(f_a_n, "alpha")
            self.id.setGuess(f_b_n, "beta")

            # comparing block: will answer the "Are we there yet?" question
            if abs(E_old - E_new) < self.id.converge:
                convergence = True


            # maintenance block: keeps everything going
            print(f"iteration: {itercount}, E_tot: {E_total: .8f}, E_elek: {E_new: .8f}, deltaE: {E_new - E_old: .8f}")
            E_old = E_new

            itercount += 1
        
        return E_total
    
    def setElectrons(self, alpha, beta):
        """sets the amount electrons of each spin"""
        self.id.alpha = alpha
        self.id.beta = beta

In [14]:
psi4.set_options({"BASIS": "cc-pvdz", 'reference': 'cuhf'})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-20)
h1.iteratinator()

iteration: 0, E_tot: -3.07773556, E_elek: -4.66526719, deltaE: -4.66526719
iteration: 1, E_tot: -1.34539207, E_elek: -2.93292370, deltaE:  1.73234349
iteration: 2, E_tot: -1.48343662, E_elek: -3.07096825, deltaE: -0.13804455
iteration: 3, E_tot: -1.49962176, E_elek: -3.08715339, deltaE: -0.01618514
iteration: 4, E_tot: -1.50223131, E_elek: -3.08976295, deltaE: -0.00260956
iteration: 5, E_tot: -1.50281605, E_elek: -3.09034768, deltaE: -0.00058474
iteration: 6, E_tot: -1.50299525, E_elek: -3.09052688, deltaE: -0.00017920
iteration: 7, E_tot: -1.50306196, E_elek: -3.09059359, deltaE: -0.00006671
iteration: 8, E_tot: -1.50308948, E_elek: -3.09062112, deltaE: -0.00002752
iteration: 9, E_tot: -1.50310151, E_elek: -3.09063314, deltaE: -0.00001203
iteration: 10, E_tot: -1.50310696, E_elek: -3.09063859, deltaE: -0.00000545
iteration: 11, E_tot: -1.50310950, E_elek: -3.09064113, deltaE: -0.00000254
iteration: 12, E_tot: -1.50311071, E_elek: -3.09064234, deltaE: -0.00000121
iteration: 13, E_tot: 

iteration: 113, E_tot: -1.50311186, E_elek: -3.09064350, deltaE: -0.00000000
iteration: 114, E_tot: -1.50311186, E_elek: -3.09064350, deltaE: -0.00000000
iteration: 115, E_tot: -1.50311186, E_elek: -3.09064350, deltaE:  0.00000000
iteration: 116, E_tot: -1.50311186, E_elek: -3.09064350, deltaE: -0.00000000
iteration: 117, E_tot: -1.50311186, E_elek: -3.09064350, deltaE:  0.00000000


-1.5031118631777187

In [8]:
psi4.set_options({"basis":"STO-3G", 'reference':'cuhf'})
allyl = ConstrainedMolecule("""
0 2
H
C 1 r2
C 2 r3 1 a3
C 2 r3 1 a3 3 180.
H 3 r5 2 a5 1 0.
H 4 r5 2 a5 1 0.
H 3 r7 2 a7 1 180.
H 4 r7 2 a7 1 180.

r2=1.08424658
r3=1.40526604
r5=1.08095381
r7=1.08131649
a3=117.99450641
a5=121.41544408
a7=121.21891262
symmetry c1

""")
allyl.id.setConvergence(1e-12)
a = allyl.id.alpha
b = allyl.id.beta
tot = a + b -1

allyl.setElectrons(tot//2 + tot%2, tot//2)
allyl.iteratinator()

iteration: 0, E_tot: -220.74589426, E_elek: -285.09922572, deltaE: -285.09922572
iteration: 1, E_tot: -101.28247061, E_elek: -165.63580207, deltaE:  119.46342365
iteration: 2, E_tot: -119.19963780, E_elek: -183.55296926, deltaE: -17.91716719
iteration: 3, E_tot: -112.76779541, E_elek: -177.12112687, deltaE:  6.43184239
iteration: 4, E_tot: -114.24306055, E_elek: -178.59639201, deltaE: -1.47526514
iteration: 5, E_tot: -114.33545897, E_elek: -178.68879043, deltaE: -0.09239842
iteration: 6, E_tot: -115.17074866, E_elek: -179.52408012, deltaE: -0.83528969
iteration: 7, E_tot: -114.60853473, E_elek: -178.96186619, deltaE:  0.56221393
iteration: 8, E_tot: -114.88318365, E_elek: -179.23651511, deltaE: -0.27464892
iteration: 9, E_tot: -114.77474352, E_elek: -179.12807498, deltaE:  0.10844013
iteration: 10, E_tot: -114.81838531, E_elek: -179.17171677, deltaE: -0.04364179
iteration: 11, E_tot: -114.80072186, E_elek: -179.15405332, deltaE:  0.01766346
iteration: 12, E_tot: -114.80771748, E_elek: 

-114.80563585642133

In [9]:
psi4.energy("scf")

-115.00929345796257

In [4]:
psi4.set_options({"basis":"cc-pvdz",'reference':'cuhf'})
m2 = ConstrainedMolecule("""
O
H 1 0.96
H 1 0.96 2 104.5
symmetry c1
""")
m2.id.setConvergence(1e-20)
m2.iteratinator()

/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":


iteration: 0, E_tot: -130.39723662, E_elek: -139.56542991, deltaE: -139.56542991
iteration: 1, E_tot: -59.41525394, E_elek: -68.58344724, deltaE:  70.98198267
iteration: 2, E_tot: -89.51296185, E_elek: -98.68115514, deltaE: -30.09770790
iteration: 3, E_tot: -66.79324033, E_elek: -75.96143363, deltaE:  22.71972151
iteration: 4, E_tot: -82.14507691, E_elek: -91.31327020, deltaE: -15.35183657
iteration: 5, E_tot: -72.25642005, E_elek: -81.42461335, deltaE:  9.88865685
iteration: 6, E_tot: -78.28128946, E_elek: -87.44948275, deltaE: -6.02486941
iteration: 7, E_tot: -74.71561601, E_elek: -83.88380931, deltaE:  3.56567344
iteration: 8, E_tot: -76.79184817, E_elek: -85.96004147, deltaE: -2.07623216
iteration: 9, E_tot: -75.58519007, E_elek: -84.75338336, deltaE:  1.20665811
iteration: 10, E_tot: -76.28267501, E_elek: -85.45086830, deltaE: -0.69748494
iteration: 11, E_tot: -75.87889284, E_elek: -85.04708613, deltaE:  0.40378217
iteration: 12, E_tot: -76.11217617, E_elek: -85.28036946, deltaE: 

iteration: 110, E_tot: -76.02665366, E_elek: -85.19484696, deltaE: -0.00000000
iteration: 111, E_tot: -76.02665366, E_elek: -85.19484696, deltaE:  0.00000000
iteration: 112, E_tot: -76.02665366, E_elek: -85.19484696, deltaE: -0.00000000
iteration: 113, E_tot: -76.02665366, E_elek: -85.19484696, deltaE:  0.00000000
iteration: 114, E_tot: -76.02665366, E_elek: -85.19484696, deltaE:  0.00000000


-76.02665366188843

In [5]:
mol = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
symmetry c1
""")
psi4.energy('scf')

-76.0266327345505

In [10]:
m3 = ConstrainedMolecule("""pubchem:methane""")
m3.iteratinator()

	Searching PubChem database for methane (single best match returned)
	Found 1 result(s)
iteration: 0, E_tot: -68.89875604, E_elek: -82.30706146, deltaE: -82.30706146
iteration: 1, E_tot: -35.78344979, E_elek: -49.19175521, deltaE:  33.11530625
iteration: 2, E_tot: -40.47161031, E_elek: -53.87991573, deltaE: -4.68816053
iteration: 3, E_tot: -39.58781908, E_elek: -52.99612450, deltaE:  0.88379123
iteration: 4, E_tot: -39.75174659, E_elek: -53.16005201, deltaE: -0.16392750
iteration: 5, E_tot: -39.72203067, E_elek: -53.13033609, deltaE:  0.02971592
iteration: 6, E_tot: -39.72741518, E_elek: -53.13572059, deltaE: -0.00538451
iteration: 7, E_tot: -39.72643194, E_elek: -53.13473736, deltaE:  0.00098323
iteration: 8, E_tot: -39.72660936, E_elek: -53.13491478, deltaE: -0.00017741
iteration: 9, E_tot: -39.72657680, E_elek: -53.13488221, deltaE:  0.00003256
iteration: 10, E_tot: -39.72658263, E_elek: -53.13488805, deltaE: -0.00000584
iteration: 11, E_tot: -39.72658155, E_elek: -53.13488697, delt

-39.726581742671634

In [11]:
psi4.energy("scf")

-39.7267287296376

In [ ]:
f_a = h1.id.displayFockMatrix('alpha')
c = eigh(f_a, h1.id.overlap)[1]
a = h1.id.getDensityMatrix("alpha")
b = h1.id.getDensityMatrix("beta")
p = (a + b)/2
e = c.T.dot(p).dot(c)
f1, f = eigh(e)
f = f[:, ::-1]
f_trans = c.T.dot(f_a).dot(c)
f_no = f.T.dot(f_trans).dot(f)
q = f.T.dot(e).dot(f)
q[abs(q) < 1e-10] = 0
q

This value does not seem correct (UHF gives about -1.506 H for the total energy). This might be the case because the Fock operator depends on the two electron integrals have not been transformed to the correct basis yet. Can this be the case?


In [ ]:
psi4.set_options({"BASIS": "cc-pvdz"})
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")

In [ ]:
s = h_test.id.overlap
s_eigh = eigh(s)
s_diag = np.diag(s_eigh[0]**(-1/2))
x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
x.T.dot(s).dot(x)

In [ ]:
a = h_test.id.getDensityMatrix('alpha')
b = h_test.id.getDensityMatrix('beta')
x_i = np.linalg.inv(x)
x_i_t = np.linalg.inv(x.T)
a_trans = x_i.dot(a).dot(x_i_t)
b_trans = x_i.dot(b).dot(x_i_t)
p = (a_trans + b_trans)/2
c = eigh(p)
c

Up untill here everything should be correct (density matrices are idempotent, p matrix has correct eigenvalues). The area below is still under construction.

In [ ]:
f_a = h_test.id.displayFockMatrix('alpha')
f_b = h_test.id.displayFockMatrix('beta')

In [ ]:
f_a_trans = x.dot(f_a).dot(x.T)
f_a_no = c[1].T.dot(f_a_trans).dot(c[1])
f_b_trans = x.dot(f_b).dot(x.T)
f_b_no = c[1].T.dot(f_b_trans).dot(c[1])

This expression seems a bit too simple. Nevertheless, according to Tsuchimochi & Scuseria it should hold.

In [ ]:
f_cs = (f_a_no + f_b_no)/2

In [ ]:
a = h_test.id.getDensityMatrix("alpha")
b = h_test.id.getDensityMatrix("beta")
a - b

In [ ]:
f_b = h_test.id.displayFockMatrix("beta")
f_a = h_test.id.displayFockMatrix("alpha")

In [ ]:
f_cs = (f_a + f_b)/2

In [ ]:
a = h_test.id.getDensityMatrix("alpha")
b = h_test.id.getDensityMatrix("beta")
p = (a + b)/2

In [ ]:
c1, c = eigh(f_a, h_test.id.overlap)
c_inv = np.linalg.inv(c)
c_inv

In [ ]:
d = c_inv.T.dot(p).dot(c_inv)
o1, o = eigh(d)
o = o[:, ::-1]
j = o.T.dot(d).dot(o)
j[abs(j) < 1e-10] = 0
j

In [ ]:
f_trans = c.T.dot(f_a).dot(c)
f_trans[abs(f_trans) < 1e-10] = 0
f_trans

In [ ]:
a_trans = c_inv.dot(a).dot(c_inv.T)
a_no = o.T.dot(a_trans).dot(o)
a_no[abs(a_no) < 1e-10] = 0
a_no


In [ ]:
o.dot(a_no).dot(o.T)

In [ ]:
a_trans

In [ ]:
y = eigh(p, h_test.id.overlap)

In [ ]:
n = np.array([[1,2], [2,3]])
v = eigh(n)
v

In [ ]:
v[1].dot(n).dot(v[1].T)

In [ ]:
o.T.dot(o)